<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/%D0%A1%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B8_yargy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
!wget = https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

--2020-01-19 14:39:48--  http://=/
Resolving = (=)... failed: Name or service not known.
wget: unable to resolve host address ‘=’
--2020-01-19 14:39:48--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2939556 (2.8M) [text/plain]
Saving to: ‘pristavki.csv’

pristavki.csv       100%[===================>]   2.80M  16.0MB/s    in 0.2s    

2020-01-19 14:39:49 (16.0 MB/s) - ‘pristavki.csv’ saved [2939556/2939556]

FINISHED --2020-01-19 14:39:49--
Total wall clock time: 0.4s
Downloaded: 1 files, 2.8M in 0.2s (16.0 MB/s)


In [0]:
pd.set_option('display.max_colwidth', -1)
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
!pip install yargy
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

     |████████████████████████████████| 51kB 2.3MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 7.1MB 7.7MB/s 


In [0]:
Game = fact(
    'Game',
    ['name', 'version']
)

In [0]:
mortal = rule(
    morph_pipeline(['мортал ', 'мортал комбат', 'mortal combat', 'mortal kombat']).interpretation(Game.name.const("Mortal Kombat")),
    morph_pipeline(['X', 'XL', '11']).interpretation(Game.version).optional())

assassin = rule(
    morph_pipeline(['асасин', 'асассин', 'assassin', 'Assassins Creed', 'Ассассин крид']).interpretation(Game.name.const("Assassin's creed")),
    morph_pipeline(['2', '3', '4']).interpretation(Game.version).optional()) 

GTA_VERSIONS = {
    '5': 'V',
    'V': 'V',
    'v': 'V',
    '4': 'IV',
}

gta = rule(
    morph_pipeline(['гта', 'gta', 'gta5', 'гта5']).interpretation(Game.name.const('GTA')),
    caseless_pipeline(GTA_VERSIONS).interpretation(Game.version.normalized().custom(GTA_VERSIONS.get)).optional()) 

fifa = rule(
    morph_pipeline(['fifa', 'фифа']).interpretation(Game.name.const("FIFA")),
    morph_pipeline(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19']).interpretation(Game.version).optional()) 

vedmak = rule(
    morph_pipeline(['Ведьмак', 'ведьмак2', 'ведьмак3']).interpretation(Game.name.const('Ведьмак')),
    morph_pipeline(['1', '2', '3']).interpretation(Game.version).optional()) 

GAME = or_(gta, assassin, fifa, vedmak, mortal).interpretation(Game)
parser = Parser(GAME)

In [0]:
matches = []

for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)

print(len(matches))

1728


In [0]:
for m in matches[:30]:
    if m.version:
      print(m.name, m.version)
    else:
      print(m.name)

FIFA 15
GTA V
Assassin's creed 4
GTA IV
Mortal Kombat
FIFA 16
GTA V
FIFA
Assassin's creed 4
FIFA 11
FIFA 15
GTA V
GTA V
FIFA
GTA V
Mortal Kombat
GTA V
Ведьмак
Assassin's creed
GTA V
GTA V
FIFA
GTA V
GTA V
Mortal Kombat
FIFA 16
Assassin's creed
FIFA 13
FIFA 16
FIFA


Результаты:

1) Наименований игр очень много, одна игра может иметь более 10 версий и перечислить их все в правилах довольно затруднительно, особенно с учетом всех возможных вариантов написания

2) Распространенная у авторов объявлений ошибка - слитное написание названия игры и номера версии. Мы можем указать такие случаи в правилах для извлечения названий игр, но номер версии при этом теряется

3) Поиск возможных вариантов усложняет частое написание названий на английском языке русскими буквами - в таком случае орфография становится сложно предсказуемой